In [1]:
## import pandas as pd
import pandas as pd
import requests
#from graph import *
import plotly.graph_objects as go
import math
from scipy import stats
import string
import numpy as np
import time
from scipy.stats import zscore
import sys
import os
def get_index():
    teams_response = requests.get("https://api.pbpstats.com/get-teams/nba")
    teams = teams_response.json()
    team_dict = {}
    for team in teams['teams']:
        team_dict[team['text']] = team['id']
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.183',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Cache-Control': 'max-age=0',
    }

    players_response = requests.get("https://api.pbpstats.com/get-all-players-for-league/nba",headers=headers)
    #print(players_response.json())
    players = players_response.json()["players"]
    #print(players)
    player_dict = dict([(player.lower(),num) for num,player in players.items()])
  
    return player_dict,team_dict


def playerpull(player_id,team_id,season,opp=False,ps = False):
    term = "Team"
    if opp==True:
        term="Opponent"

    s_type="Regular Season"
    if ps==True:
        S_type="Playoffs"

    wowy_url = "https://api.pbpstats.com/get-wowy-stats/nba"
    wowy_params = {
        "0Exactly1OnFloor": player_id, # Player on
        "TeamId": team_id, # Golden State Warriors
        
        "Season": season,
        "SeasonType": s_type,
        "Type": term

    }
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.183',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Cache-Control': 'max-age=0',}
    wowy_response = requests.get(wowy_url, params=wowy_params,headers=headers)

    wowy = wowy_response.json()
    #print(wowy)
    player_stats_on = wowy["single_row_table_data"]

    
    #print(player_stats_on)
    wowy_url = "https://api.pbpstats.com/get-wowy-stats/nba"
    headers = {
        'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Mobile/15E148 Safari/604.1',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
    }

    wowy_params = {
        "0Exactly0OnFloor": player_id,# Player off
        "TeamId": team_id, # Golden State Warriors
        "Season":  season,
        "SeasonType": s_type,
        "Type": term# Player stats,
    }
    time.sleep(2)

    wowy_response = requests.get(wowy_url, params=wowy_params,headers=headers)
    time.sleep(2)

    wowy = wowy_response.json()
    player_stats_off = wowy["single_row_table_data"]

    data_columns = ['AtRimFGM','AtRimFGA','AtRimAssists','Turnovers','FTA',
    'OffPoss','DefPoss','TwoPtAssists','Steals','BadPassSteals','FG2M','FG#M','FTPoints',
    'ThreePtAssists','FG3A','OpponentPoints','Points','SecondChancePoints','FG2A','LiveBallTurnovers',
                                 'TsPct','Fg3Pct','AtRimFG3AFrequency','Pace','AtRimFrequency',
                                  'SecondsPerPossOff','SecondsPerPossDef','Minutes',
                'DefAtRimReboundPct','DefFGReboundPct','AtRimAccuracy','SecondChancePoints','SecondChanceFG2A','SecondChanceAtRimFGM','PenaltyOffPoss','PenaltyDefPoss'
                'OffFGReboundPct']

    df = pd.DataFrame(player_stats_on,index = [0])

    #df1_columns = [c for c in data_columns if c in df.columns]
    #df=df[data_columns]
    df['player_on']=True
    
    
    df2 = pd.DataFrame(player_stats_off,index = [0])
    #df2_columns = [c for c in data_columns if c in df2.columns]
    #df2=df2[data_columns]
    df2['player_on']=False

    return pd.concat([df,df2])


years = [year for year in range(2001 ,2025)]
def pull_onoff(years,opp=False,ps=False):
    
    count=0
    opp_string=''
    if opp==True:
        opp_string='vs'
    pstring=''
    if ps==True:
        pstring='ps'
    player_index = pd.read_csv('https://raw.githubusercontent.com/gabriel1200/shot_data/master/player_index'+pstring+'.csv')
    player_index = player_index[player_index.year>2000]
    for year in years:
        

        
        season_index=player_index[player_index.year==year].reset_index(drop=True)

        old_file = (str(year))+opp_string+pstring+'.csv'
        print(old_file)
        if os.path.exists(old_file):
            old_df = pd.read_csv(old_file)
            old_df['total_id'] = old_df['player_id']+old_df['team_id']
            season_index['total_id']  = season_index['PLAYER_ID']+season_index['TEAM_ID']
            total_ids = old_df['total_id'].tolist()
            season_index = season_index[~season_index.total_id.isin(total_ids)]
            season_index.drop(columns='total_id',inplace=True)
            
        print(len(season_index))

        season = str(year-1)+'-'+str(year)[-2:]
    
        frames=[]
        fail_list=[]
        for player_id,team_id in zip(season_index.PLAYER_ID,season_index.TEAM_ID):
            try:
               
        
                df= playerpull(player_id,team_id,season,opp=opp,ps=ps)
                df =df.reset_index(drop=True)
                df['player_id']=player_id
                df['team_id']=team_id
                df['year']=year
                df['season']=season
                df['player_vs']=opp
                frames.append(df)
                count+=1
            except Exception as e:
                print(e)
                print(str(player_id) +' not found')
                fail_list.append(player_id)
            if count %25==1:
                print(count)
               
                year_frame = pd.concat(frames)
                if os.path.exists(old_file):
                    old_df =pd.read_csv(old_file)
                    year_frame = pd.concat([old_df,year_frame])
                    del old_df
                
                year_frame.to_csv(str(year)+opp_string+pstring+'.csv',index=False)
                del(year_frame)
            
        if len(frames)>0:   
        
            year_frame = pd.concat(frames)
            if os.path.exists(old_file):
                old_df =pd.read_csv(old_file)
                year_frame = pd.concat([old_df,year_frame])
            
            print('Year Completed, Saving')
            print(fail_list)
            year_frame.to_csv(str(year)+opp_string+pstring+'.csv',index=False)
        else:
            print('Year already Saved')

      
        
pull_onoff(years,opp=True,ps=False) 


/tmp/ipykernel_5330/3599450204.py:146: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  player_index = pd.read_csv('https://raw.githubusercontent.com/gabriel1200/shot_data/master/player_index'+pstring+'.csv')


2001vs.csv
0
Year already Saved
2002vs.csv
0
Year already Saved
2003vs.csv
0
Year already Saved
2004vs.csv
0
Year already Saved
2005vs.csv
0
Year already Saved
2006vs.csv
0
Year already Saved
2007vs.csv
0
Year already Saved
2008vs.csv
0
Year already Saved
2009vs.csv
0
Year already Saved
2010vs.csv
0
Year already Saved
2011vs.csv
0
Year already Saved
2012vs.csv
0
Year already Saved
2013vs.csv
0
Year already Saved
2014vs.csv
0
Year already Saved
2015vs.csv
0
Year already Saved
2016vs.csv
0
Year already Saved
2017vs.csv
0
Year already Saved
2018vs.csv
0
Year already Saved
2019vs.csv
0
Year already Saved
2020vs.csv
0
Year already Saved
2021vs.csv
0
Year already Saved
2022vs.csv
0
Year already Saved
2023vs.csv
0
Year already Saved
2024vs.csv
0
Year already Saved


In [2]:
import pandas as pd
import pandas as pd
import requests
#from graph import *
import plotly.graph_objects as go
import math
from scipy import stats
import string
import numpy as np

import time
from scipy.stats import zscore
import sys
import os
def get_index():
    teams_response = requests.get("https://api.pbpstats.com/get-teams/nba")
    teams = teams_response.json()
    team_dict = {}
    for team in teams['teams']:
        team_dict[team['text']] = team['id']
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}

    players_response = requests.get("https://api.pbpstats.com/get-all-players-for-league/nba",headers=headers)
    #print(players_response.json())
    players = players_response.json()["players"]
    #print(players)
    player_dict = dict([(player.lower(),num) for num,player in players.items()])
  
    return player_dict,team_dict


def playerpull(player_id,team_id,season,opp=False,ps = False):
    term = "Team"
    if opp==True:
        term="Opponent"

    s_type="Regular Season"
    if ps==True:
        S_type="Playoffs"

    wowy_url = "https://api.pbpstats.com/get-wowy-stats/nba"
    wowy_params = {
        "0Exactly1OnFloor": player_id, # Player on
        "TeamId": team_id, # Golden State Warriors
        
        "Season": season,
        "SeasonType": s_type,
        "Type": term

    }
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
    wowy_response = requests.get(wowy_url, params=wowy_params,headers=headers)
    wowy = wowy_response.json()
    #print(wowy)
    player_stats_on = wowy["single_row_table_data"]

    
    #print(player_stats_on)
    wowy_url = "https://api.pbpstats.com/get-wowy-stats/nba"

    wowy_params = {
        "0Exactly0OnFloor": player_id,# Player off
        "TeamId": team_id, # Golden State Warriors
        "Season":  season,
        "SeasonType": s_type,
        "Type": term# Player stats,
    }
    time.sleep(.5)
    wowy_response = requests.get(wowy_url, params=wowy_params,headers=headers)
    time.sleep(1)
    wowy = wowy_response.json()
    player_stats_off = wowy["single_row_table_data"]

    data_columns = ['AtRimFGM','AtRimFGA','AtRimAssists','Turnovers','FTA',
    'OffPoss','DefPoss','TwoPtAssists','Steals','BadPassSteals','FG2M','FG#M','FTPoints',
    'ThreePtAssists','FG3A','OpponentPoints','Points','SecondChancePoints','FG2A','LiveBallTurnovers',
                                 'TsPct','Fg3Pct','AtRimFG3AFrequency','Pace','AtRimFrequency',
                                  'SecondsPerPossOff','SecondsPerPossDef','Minutes',
                'DefAtRimReboundPct','DefFGReboundPct','AtRimAccuracy','SecondChancePoints','SecondChanceFG2A','SecondChanceAtRimFGM','PenaltyOffPoss','PenaltyDefPoss'
                'OffFGReboundPct']

    df = pd.DataFrame(player_stats_on,index = [0])

    #df1_columns = [c for c in data_columns if c in df.columns]
    #df=df[data_columns]
    df['player_on']=True
    
    
    df2 = pd.DataFrame(player_stats_off,index = [0])
    #df2_columns = [c for c in data_columns if c in df2.columns]
    #df2=df2[data_columns]
    df2['player_on']=False

    return pd.concat([df,df2])


years = [year for year in range(2001 ,2025)]
def pull_onoff(years,opp=False,ps=False):
    
    count=0
    opp_string=''
    if opp==True:
        opp_string='vs'
    pstring=''
    if ps==True:
        pstring='ps'
    player_index = pd.read_csv('https://raw.githubusercontent.com/gabriel1200/shot_data/master/player_index'+pstring+'.csv')
    player_index = player_index[player_index.year>2000]
    for year in years:
        

        
        season_index=player_index[player_index.year==year].reset_index(drop=True)

        old_file = (str(year))+opp_string+pstring+'.csv'
        print(old_file)
        if os.path.exists(old_file):
            old_df = pd.read_csv(old_file)
            old_df['total_id'] = old_df['player_id']+old_df['team_id']
            season_index['total_id']  = season_index['PLAYER_ID']+season_index['TEAM_ID']
            total_ids = old_df['total_id'].tolist()
            season_index = season_index[~season_index.total_id.isin(total_ids)]
            season_index.drop(columns='total_id',inplace=True)
            
        print(len(season_index))

        season = str(year-1)+'-'+str(year)[-2:]
    
        frames=[]
        fail_list=[]
        for player_id,team_id in zip(season_index.PLAYER_ID,season_index.TEAM_ID):
            try:
               
        
                df= playerpull(player_id,team_id,season,opp=opp,ps=ps)
                df =df.reset_index(drop=True)
                df['player_id']=player_id
                df['team_id']=team_id
                df['year']=year
                df['season']=season
                df['player_vs']=opp
                frames.append(df)
                count+=1
            except Exception as e:
                print(e)
                print(str(player_id) +' not found')
                fail_list.append(player_id)
            if count %25==1:
                print(count)
               
                year_frame = pd.concat(frames)
                if os.path.exists(old_file):
                    old_df =pd.read_csv(old_file)
                    year_frame = pd.concat([old_df,year_frame])
                    del old_df
                
                year_frame.to_csv(str(year)+opp_string+pstring+'.csv',index=False)
                del(year_frame)
            
        if len(frames)>0:   
        
            year_frame = pd.concat(frames)
            if os.path.exists(old_file):
                old_df =pd.read_csv(old_file)
                year_frame = pd.concat([old_df,year_frame])
            
            print('Year Completed, Saving')
            print(fail_list)
            year_frame.to_csv(str(year)+opp_string+pstring+'.csv',index=False)
        else:
            print('Year already Saved')

      
        
pull_onoff(years,opp=False,ps=False) 


/tmp/ipykernel_5330/387320399.py:107: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  player_index = pd.read_csv('https://raw.githubusercontent.com/gabriel1200/shot_data/master/player_index'+pstring+'.csv')


2001.csv
0
Year already Saved
2002.csv
0
Year already Saved
2003.csv
0
Year already Saved
2004.csv
0
Year already Saved
2005.csv
0
Year already Saved
2006.csv
0
Year already Saved
2007.csv
0
Year already Saved
2008.csv
0
Year already Saved
2009.csv
0
Year already Saved
2010.csv
0
Year already Saved
2011.csv
0
Year already Saved
2012.csv
0
Year already Saved
2013.csv
0
Year already Saved
2014.csv
0
Year already Saved
2015.csv
0
Year already Saved
2016.csv
0
Year already Saved
2017.csv
0
Year already Saved
2018.csv
0
Year already Saved
2019.csv
0
Year already Saved
2020.csv
0
Year already Saved
2021.csv
0
Year already Saved
2022.csv
0
Year already Saved
2023.csv
0
Year already Saved
2024.csv
0
Year already Saved


In [3]:
df = pd.read_csv('player_team_wowy/2001.csv')
df
    

,SecondsPlayed,Minutes,PlusMinus,OffPoss,DefPoss,PenaltyOffPoss,PenaltyDefPoss,SecondChanceOffPoss,TotalPoss,AtRimFGM,...,BlockedArc3,BlockedCorner3,player_id,team_id,year,season,player_vs,3pt And 1 Free Throw Trips,FoulsDrawn,Offensive Fouls Drawn
0,108480.0,1808,-263.0,3485,3477,816.0,971.0,469.0,6962,499.0,...,NaN,NaN,673,1610612737,2001,2000-01,False,NaN,NaN,NaN
1,128280.0,2138,-164.0,4115,4129,1112.0,1295.0,532.0,8244,578.0,...,2.0,1.0,673,1610612737,2001,2000-01,False,NaN,NaN,NaN
2,33.0,1,NaN,1,1,1.0,1.0,NaN,2,NaN,...,NaN,NaN,1950,1610612737,2001,2000-01,False,NaN,NaN,NaN
3,236727.0,3945,-427.0,7599,7605,1927.0,2265.0,1001.0,15204,1077.0,...,2.0,1.0,1950,1610612737,2001,2000-01,False,NaN,NaN,NaN
4,16736.0,279,-62.0,544,550,211.0,265.0,64.0,1094,71.0,...,2.0,NaN,1533,1610612737,2001,2000-01,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,236163.0,3936,180.0,7383,7367,2167.0,1847.0,990.0,14750,1061.0,...,7.0,2.0,197,1610612766,2001,2000-01,False,NaN,NaN,NaN
976,739.0,12,-6.0,26,26,23.0,15.0,5.0,52,7.0,...,NaN,NaN,2121,1610612766,2001,2000-01,False,NaN,NaN,NaN
977,237821.0,3964,178.0,7436,7418,2168.0,1862.0,993.0,14854,1061.0,...,7.0,2.0,2121,1610612766,2001,2000-01,False,NaN,NaN,NaN
978,11925.0,199,-5.0,371,377,144.0,144.0,52.0,748,46.0,...,1.0,NaN,1906,1610612766,2001,2000-01,False,NaN,NaN,NaN


In [5]:
for col in df:
    print(col)

SecondsPlayed
Minutes
PlusMinus
OffPoss
DefPoss
PenaltyOffPoss
PenaltyDefPoss
SecondChanceOffPoss
TotalPoss
AtRimFGM
AtRimFGA
SecondChanceAtRimFGM
SecondChanceAtRimFGA
PenaltyAtRimFGM
PenaltyAtRimFGA
ShortMidRangeFGM
ShortMidRangeFGA
LongMidRangeFGM
LongMidRangeFGA
Corner3FGM
Corner3FGA
SecondChanceCorner3FGM
SecondChanceCorner3FGA
PenaltyCorner3FGM
PenaltyCorner3FGA
Arc3FGM
Arc3FGA
SecondChanceArc3FGM
SecondChanceArc3FGA
PenaltyArc3FGM
PenaltyArc3FGA
FG2M
FG2A
FG3M
FG3A
FtPoints
Points
OpponentPoints
SecondChanceFG2M
SecondChanceFG2A
SecondChanceFG3M
SecondChanceFG3A
SecondChanceFtPoints
SecondChancePoints
PenaltyFG2M
PenaltyFG2A
PenaltyFG3M
PenaltyFG3A
PenaltyFtPoints
PenaltyPoints
PtsAssisted2s
PtsUnassisted2s
PtsAssisted3s
PtsUnassisted3s
PtsPutbacks
HeaveAttempts
NonHeaveArc3FGA
NonHeaveArc3FGM
Fg2aBlocked
Fg3aBlocked
TwoPtAssists
ThreePtAssists
Assists
Arc3Assists
Corner3Assists
AtRimAssists
ShortMidRangeAssists
LongMidRangeAssists
AssistPoints
OffThreePtRebounds
OffTwoPtRebounds